In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


https://api.meraki.com/api/v0/networks/N_711005791171205780/cameras/Q2JV-BY67-ABC8/snapshot'

network : N_711005791171205780

In [ ]:
 #! pip install boto3==1.12.37 botocore==1.15.37 certifi==2020.4.5.1 chardet==3.0.4 docutils==0.15.2 idna==2.9 jmespath==0.9.5 paho-mqtt==1.5.0 python-dateutil==2.8.1 requests==2.23.0 s3transfer==0.3.3 six==1.14.0 urllib3==1.25.8

In [ ]:
#!cat '/content/drive/My Drive/SIH 2020/credentials.ini'

In [69]:
import os
import configparser
import sys
import json
import requests
import boto3
import paho.mqtt.client as mqtt
from PIL import Image
import requests
from io import BytesIO
import datetime
import pytz
os.getcwd()

'/content'

In [70]:
BASE_PATH = '/content/drive/My Drive/SIH 2020'


In [87]:
def iso(ts):
  # return time.strftime("%Y %d %b %H:%M:%S +0000", time.localtime(ts))
  d = datetime.datetime.utcfromtimestamp(ts/1000).isoformat()
  return d[:-7]+'Z'

In [88]:
def url_to_img(url):
  response = requests.get(url)
  img = Image.open(BytesIO(response.content))
  return img

In [89]:
def on_connect(mq_client, userdata, flags, result_code):
    """The callback for when the client receives a CONNACK response from the server"""
    print(f'Connected with result code {result_code}')
    serial = userdata['MV_SERIAL']
    # Subscribing in on_connect() means that if we lose the connection and
    # reconnect then subscriptions will be renewed.
    mq_client.subscribe(f'/merakimv/{serial}/raw_detections')
    #client.subscribe(f'/merakimv/{serial}/light')

In [118]:
res_arr = []
def on_message(client, userdata, msg):
    """When a PUBLISH message is received from the server, get a
    URL to analyse"""
    #triggers image analysis when incoming MQTT data is detected
    # print(client, userdata, msq)
    obj = msg.payload.decode("utf-8")
    res = json.loads(obj)
    print(res)
    objs = res['objects']
    person_count = len(objs)
    ts = res['ts']
    # print(ts, person_count)
    if(person_count>=0):
      ts = iso(ts)
      # s_url = get_meraki_snapshots(API_KEY, NET_ID, ts)
      # res['url'] = s_url
      res_arr.append(ts)
      print("Item Saved with person > 0 and timestamp: ", ts)

In [99]:
# def analyze(ts):
#     """periodially takes snap URL from Meraki, sends to AWS rekognition"""
#     flag = True
#     if flag:
#         # print("Request Snapshot URL")
#         #get the URL of a snapshot from our camera
#         ts_arr.append(ts) 

In [100]:
def get_meraki_snapshots(api_key, net_id, time=None):
    """Get devices of network"""
    headers = {
        'X-Cisco-Meraki-API-Key': api_key,
        'Content-Type': 'application/json'
        # issue where this is only needed if timestamp specified
    }
    response = session.get(f'https://api.meraki.com/api/v0/networks/{net_id}/devices',
                           headers=headers)
    devices = response.json()
    #filter for MV cameras:
    cameras = [device for device in devices if device['model'][:2] == 'MV']
    # Assemble return data
    for camera in cameras:
        #filter for serial number provided
        if camera["serial"] == MV_SERIAL:
            # Get snapshot link
            if time:
                headers['Content-Type'] = 'application/json'
                response = session.post(
                    f'https://api.meraki.com/api/v0/networks/{net_id}/cameras/{camera["serial"]}/snapshot',
                    headers=headers,
                    data=json.dumps({'timestamp': time}))
            else:
                response = session.post(
                    f'https://api.meraki.com/api/v0/networks/{net_id}/cameras/{camera["serial"]}/snapshot',
                    headers=headers)

            # Possibly no snapshot if camera offline, photo not retrievable, etc.
            print(response.json())
            if response.ok:
                snapshot_url = response.json()['url']
                return snapshot_url
            else:
              return None
        else:
          return None


In [101]:
def gather_credentials():
    """Gather Meraki credentials"""
    conf_par = configparser.ConfigParser()
    try:
        conf_par.read(os.path.join(BASE_PATH, 'credentials.ini'))
        # conf_par.read('credentials.ini')
        cam_key = conf_par.get('meraki', 'key')
        net_id = conf_par.get('meraki', 'network')
        mv_serial = conf_par.get('sense', 'serial')
        server_ip = conf_par.get('server', 'ip')
    except:
        print('Missing credentials or input file!')
        sys.exit(2)
    return cam_key, net_id, mv_serial, server_ip

In [119]:
if __name__ == '__main__':

    (API_KEY, NET_ID, MV_SERIAL, SERVER_IP) = gather_credentials()
    USER_DATA = {
        'API_KEY': API_KEY,
        'NET_ID': NET_ID,
        'MV_SERIAL': MV_SERIAL,
        'SERVER_IP': SERVER_IP
    }
    session = requests.Session()
    # Start MQTT client
    client = mqtt.Client()
    client.user_data_set(USER_DATA)
    #on connection to a MQTT broker:
    client.on_connect = on_connect
    #when an MQTT message is received:
    client.on_message = on_message
    #specify the MQTT broker here
    client.connect(SERVER_IP, 1883, 300)
    client.loop_forever()

Connected with result code 0
{'ts': 1596288310223, 'objects': [{'frame': 1098983, 'oid': 0, 'x0': 0.619, 'x1': 0, 'y0': 1, 'y1': 0, 'type': 'person'}]}
Item Saved with person > 0 and timestamp:  2020-08-01T13:25:10Z
{'ts': 1596288310478, 'objects': [{'frame': 1098986, 'oid': 0, 'x0': 0.548, 'x1': 0, 'y0': 1, 'y1': 0, 'type': 'person'}]}
Item Saved with person > 0 and timestamp:  2020-08-01T13:25:10Z
{'ts': 1596288311826, 'objects': [{'frame': 1098999, 'oid': 0, 'x0': 0.184, 'x1': 0, 'y0': 0.895, 'y1': 0.0683, 'type': 'person'}, {'frame': 1098999, 'oid': 1, 'x0': 0.742, 'x1': 0.0869, 'y0': 1, 'y1': 0, 'type': 'person'}]}
Item Saved with person > 0 and timestamp:  2020-08-01T13:25:11Z
{'ts': 1596288312090, 'objects': [{'frame': 1099002, 'oid': 0, 'x0': 0.106, 'x1': 0, 'y0': 0.856, 'y1': 0.0983, 'type': 'person'}]}
Item Saved with person > 0 and timestamp:  2020-08-01T13:25:12Z
{'ts': 1596288312503, 'objects': []}
Item Saved with person > 0 and timestamp:  2020-08-01T13:25:12Z
{'ts': 1596

KeyboardInterrupt: ignored

In [120]:
res_arr

['2020-08-01T13:25:10Z',
 '2020-08-01T13:25:10Z',
 '2020-08-01T13:25:11Z',
 '2020-08-01T13:25:12Z',
 '2020-08-01T13:25:12Z',
 '2020-08-01T13:25:14Z',
 '2020-08-01T13:25:14Z']

In [122]:
url_arr = []
for ts in res_arr:
  s_url = get_meraki_snapshots(API_KEY, NET_ID, ts)
  url_arr.append(s_url)

{'url': 'https://spn18.meraki.com/stream/jpeg/snapshot/6e42acfa5b6666c9VHOGVlYmE5OTgxZTc1MmRhZDgxZDA2MGJhNzE0NTBiY2NkNzQwZTg4OWNmZDYzODMxYjhiMmM0ZjE3MTBhOWJjZbD1Saz92c1PSp2FBinzKxMnLxaTnc049p4SUp5LM5EklRJnVpH4G3Tnc-VbiHTx29jjcLlfu_WO2kaxjD2LMv9QLhQgSwNURiD0uu717p8xrdLhy9thpb8B68trVbBFkfrts10T0P96b92nHRBrrZ9I62hpeH45fuS03tDUWGcvc-UbuRx6aL9B_FGvB2rBK4u6oH_j3aEwXcDvuZoz8k2PRM8kzwWQniNyQz0rXs7E', 'expiry': 'Access to the image will expire at 2020-08-01T07:26:47-07:00'}
{'url': 'https://spn18.meraki.com/stream/jpeg/snapshot/6e42acfa5b6666c9VHYTY2YWFjYTQ0OWMwZGEzMmZlOTkwNzZiZTI0MzQ3ZjcwNWU4YTU3N2E2YTBmZTYyZjhkM2JmZTYyMzM4ODFhOKzUlsZNGt_F9JB7cfko7u979q4RPrWfN-0GsknBviXTimPjQuwJdhYJRavBpnN1ZXZ7e_6RNb-T_Qq-VEk5KE9s0UPddSjBYUaYw5uTinnd7WocEB9WRseh0Ikgtl6FilIFP_jU7orM2Ni4U5Ylb7fTr01dNJKZy96G-0QQgV08XhTNgAK3IjChJKtOzvlu7IN2_AIb1xr0PNrTOESwhStEK2MfWo1Y2roX7jUb02qx', 'expiry': 'Access to the image will expire at 2020-08-01T07:26:48-07:00'}
{'url': 'https://spn18.meraki.com/stream/jpeg/snapshot/6e42a

In [82]:
iso(1596282058612)
#2020-08-01T11:40:58Z

'2020-08-01T11:40:58Z'

In [ ]:
!curl -L -H 'X-Cisco-Meraki-API-Key: e4edb0ff642754d2b1f7146967edb38b34b3e49c' -X POST 'https://api.meraki.com/api/v0/networks/N_711005791171205780/cameras/Q2JV-BY67-ABC8/snapshot'

{"url":"https://spn18.meraki.com/stream/jpeg/snapshot/6e42acfa5b6666c9VHMDkwNjQzNWM3MWYzNWYzZjUxY2RkZjVmMjZkMjE5ZDlmOWQzNzA3ZGIxOGY0NjY5NjJjYjllYmNhMmM5MTQxNkctfyJg2ciOd28Qvi1Ny-DQL5GmqLyutZH0i4WrTwAi9RZW0YGAyvBhcuYGMqL39ylJyh32mmq5bDkHTvvqUFsjIWz8hb3okbaww--T7tTzRXnIK0gTar1RvWUqbXXeGN9KXERAgJc_WVSDYuo01fuOUwvpf4cIZJOmtXtV1hlLIDPQbqqIMJn6x8oqNrwPAu4CbixYfe03aLZWOvatqHM","expiry":"Access to the image will expire at 2020-08-01T06:58:07-07:00"}